In [1]:
# Required Modules
import numpy as np

In [66]:
class Links:

    def __init__(self, nodes, w):    # nodes = (s, d); is a tuple so as to not confuse s and d individually
        self.nodes = nodes
        self.lambda_tdc = np.ones(47, dtype = bool)

        time_slots1 = 8
        time_slots2 = 10 
        time_slots3 = 12
        self.lambda_q1 = np.ones(time_slots1, dtype = bool)    # for high sl
        self.lambda_q2 = np.ones(time_slots2, dtype = bool)    # for medium sl    
        self.lambda_q3 = np.ones(time_slots3, dtype = bool)    # for low sl

    def update_link(self, channel, ts):
        # call : links[index].update_link(channel, ts); where ts is timeslot
        #QSC = ["lambda_q1", "lambda_q2", "lambda_q3"]
        QSC = [1, 2, 3]
        if channel not in QSC or ts < 0:
            print("Invalid Update request!")

        elif channel == 1 and ts <= len(self.lambda_q1):
            self.lambda_q1[ts-1] = False
            print("The updated resources are : ", self.lambda_q1)
        elif channel == 2 and ts <= len(self.lambda_q2):
            self.lambda_q2[ts-1] = False
            print("The updated resources are : ", self.lambda_q2)
        elif channel == 3 and ts <= len(self.lambda_q3):
            self.lambda_q3[ts-1] = False
            print("The updated resources are : ", self.lambda_q3)

        else:
            print("Invalid time slot value")
            
            
    def display_info(self):
        q1_count = np.count_nonzero(self.lambda_q1) # == True
        q2_count = np.count_nonzero(self.lambda_q2) #== True
        q3_count = np.count_nonzero(self.lambda_q3) #== True
        tdc_count = np.count_nonzero(self.lambda_tdc) #== True
        print(f"Link: Nodes ={self.nodes}, lambda_tdc_count={tdc_count}, lambda_q1_count={q1_count}, lambda_q2_count={q2_count}, lambda_q3_count={q3_count}")
        
    @classmethod
    def initialize_links(cls, edges):

        all_values = [nodes for row in edges for nodes in row[:2]]    # A list of all the values/nodes in first 2 columns : s & d
        m = min(all_values)
        M = max(all_values)
        numNodes = max(all_values) - min(all_values) + 1     # Correction for the convention in indexing of nodes in the data of edges
    
        links = np.zeros([M+1, M+1], dtype = object)    # Will have redundant entries. M is the highest node index
        #link_indices = np.zeros_like(links, dtype = bool)

        #links = {}
        
        for (s, d, w) in edges:

            nodes = (s, d)
            link = cls(nodes, w)
            
            links[nodes] = link
            links[d, s] = link
            
            #print("link info")
            #link.display_info() # Working

            #print("links[nodes] info")
            #links[nodes].display_info() # Working

            #if min(all_values) == 0:
             #   (S, D) = (s, d)
            #else :
             #   (S, D) = (s-1, d-1)
            # (S, D) = (s-m, d-m)    # However, we won't be able to access the links with their indices in this case
            #links[s, d] = link
            #links[d, s] = link
            #link_indices(s, d) = True
            #link_indices(d, s) = True
                    
        indices = links != 0    # A mask that holds the location of each link
        
        return (links, indices)

In [67]:
edges = [ (1, 2, 1100),  #Edge between Node 0 and Node 1
             (2, 3, 1600), #Intermediate Edge between Node 1 and Node 2
             (1, 3, 600), # Edge between Node 0 and Node 2
             (2, 8, 1500), # Edge between Node 1 and Node 7
             (1, 4, 1000),  # Edge between Node 0 and Node 3
             (4, 5, 600), # Edge between Node 3 and Node 4
             (5, 7, 800), # Intermediate edge between Node 4 and Node 6
             (7, 8, 700), # Intermediate edge between Node 6 and Node 7
             (2, 8, 1500), # Edge between Node 1 and Node 7
             (4, 9, 1500), # Edge Betweeen Node 3 and Node 8
             (9, 12, 800), # Edge Betweeen Node 8 and Node 11
             (8, 10, 700), # Edge Betweeen Node 7 and Node 9
             (10, 12, 500), # Edge Betweeen Node 9 and Node 11
             (9, 14, 800), # Edge Betweeen Node 8 and Node 13
             (10, 14, 500), # Edge Betweeen Node 9 and Node 13
             (12, 13, 300), # Edge Betweeen Node 11 and Node 12
             (13, 14, 300), # Edge Betweeen Node 12 and Node 13
             (3, 6, 1000), # Edge Betweeen Node 2 and Node 5
             (6, 5, 1100), # Edge Betweeen Node 5 and Node 4
             (6, 13, 2000), # Edge Betweeen Node 5 and Node 12
             (6, 11, 1200), # Edge Betweeen Node 5 and Node 10
             (10, 11, 900) # Edge Betweeen Node 9 and Node 10]
        ]

(links, indices) = Links.initialize_links(edges)
#print(links, indices)

In [68]:
links[1, 2].update_link(channel = 1, ts = 6)

The updated resources are :  [ True  True  True  True  True False  True  True]


In [69]:
links[1, 2].update_link(channel = 1, ts = 11)

Invalid time slot value


In [50]:
print(indices)

[[False False False False False False False False False False False False
  False False False]
 [False False  True  True  True False False False False False False False
  False False False]
 [False  True False  True False False False False  True False False False
  False False False]
 [False  True  True False False False  True False False False False False
  False False False]
 [False  True False False False  True False False False  True False False
  False False False]
 [False False False False  True False  True  True False False False False
  False False False]
 [False False False  True False  True False False False False False  True
  False  True False]
 [False False False False False  True False False  True False False False
  False False False]
 [False False  True False False False False  True False False  True False
  False False False]
 [False False False False  True False False False False False False False
   True False  True]
 [False False False False False False False False 

In [51]:
print(links)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 <__main__.Links object at 0x00000160DAC28860>
  <__main__.Links object at 0x00000160DAC280B0> 0 0 0 0 0 0 0 0 0 0]
 [0 <__main__.Links object at 0x00000160DAC28860> 0
  <__main__.Links object at 0x00000160DAC2FEC0> 0 0 0 0
  <__main__.Links object at 0x00000160DAC2B500> 0 0 0 0 0 0]
 [0 <__main__.Links object at 0x00000160DAC2B680>
  <__main__.Links object at 0x00000160DAC2FEC0> 0 0 0
  <__main__.Links object at 0x00000160DAC2B320> 0 0 0 0 0 0 0 0]
 [0 <__main__.Links object at 0x00000160DAC280B0> 0 0 0
  <__main__.Links object at 0x00000160DAC29670> 0 0 0
  <__main__.Links object at 0x00000160DAC2AF60> 0 0 0 0 0]
 [0 0 0 0 <__main__.Links object at 0x00000160DAC29670> 0
  <__main__.Links object at 0x00000160DAC29760> 0 0 0 0 0 0 0]
 [0 0 0 <__main__.Links object at 0x00000160DAC2B320> 0
  <__main__.Links object at 0x00000160DAC292E0> 0 0 0 0 0
  <__main__.Links object at 0x00000160DAC28D40> 0
  <__main__.Links object at 0x00000160DAC2A360> 0]
 [0

In [52]:
for link in links[indices]:    # Will display double the number of actual edges, due to ordered pair
    link.display_info()

Link: Nodes =(1, 2), lambda_tdc_count=47, lambda_q1_count=8, lambda_q2_count=10, lambda_q3_count=12
Link: Nodes =(1, 3), lambda_tdc_count=47, lambda_q1_count=8, lambda_q2_count=10, lambda_q3_count=12
Link: Nodes =(1, 4), lambda_tdc_count=47, lambda_q1_count=8, lambda_q2_count=10, lambda_q3_count=12
Link: Nodes =(1, 2), lambda_tdc_count=47, lambda_q1_count=8, lambda_q2_count=10, lambda_q3_count=12
Link: Nodes =(2, 3), lambda_tdc_count=47, lambda_q1_count=8, lambda_q2_count=10, lambda_q3_count=12
Link: Nodes =(2, 8), lambda_tdc_count=47, lambda_q1_count=8, lambda_q2_count=10, lambda_q3_count=12
Link: Nodes =(1, 3), lambda_tdc_count=47, lambda_q1_count=8, lambda_q2_count=10, lambda_q3_count=12
Link: Nodes =(2, 3), lambda_tdc_count=47, lambda_q1_count=8, lambda_q2_count=10, lambda_q3_count=12
Link: Nodes =(3, 6), lambda_tdc_count=47, lambda_q1_count=8, lambda_q2_count=10, lambda_q3_count=12
Link: Nodes =(1, 4), lambda_tdc_count=47, lambda_q1_count=8, lambda_q2_count=10, lambda_q3_count=12


In [53]:
 links[indices]

array([<__main__.Links object at 0x00000160DAC28860>,
       <__main__.Links object at 0x00000160DAC2BD10>], dtype=object)

In [54]:
links[1, 2].display_info()

Link: Nodes =(1, 2), lambda_tdc_count=47, lambda_q1_count=8, lambda_q2_count=10, lambda_q3_count=12


In [55]:
index = (1, 2)
links[index].display_info()

Link: Nodes =(1, 2), lambda_tdc_count=47, lambda_q1_count=8, lambda_q2_count=10, lambda_q3_count=12
